In [0]:
from pyspark.sql import functions as F

# 1. Carregandos dados do DBFS

In [0]:
topChannels = spark.read.parquet('dbfs:/FileStore/refined/topChannels')
topChannelsMetrics = spark.read.parquet('dbfs:/FileStore/refined/topChannelsMetrics')
topVideos = spark.read.parquet('dbfs:/FileStore/refined/topVideos')
topVideosMetrics = spark.read.parquet('dbfs:/FileStore/refined/topVideosMetrics')
mostSubscribed = spark.read.parquet('dbfs:/FileStore/refined/mostSubscribed')
mostSubscribedMetrics = spark.read.parquet('dbfs:/FileStore/refined/mostSubscribedMetrics')
mostPopularChannels = spark.read.parquet('dbfs:/FileStore/refined/mostPopularChannels')
mostPopularChannelsMetrics = spark.read.parquet('dbfs:/FileStore/refined/mostPopularChannelsMetrics')
mostPopularVideos = spark.read.parquet('dbfs:/FileStore/refined/mostPopularVideos')
mostPopularVideosMetrics = spark.read.parquet('dbfs:/FileStore/refined/mostPopularVideosMetrics')
youtuberName = spark.read.parquet('dbfs:/FileStore/refined/youtuberName')

## Visualização

In [0]:
print('topChannels')
display(topChannels)
print('topChannelsMetrics')
display(topChannelsMetrics)
print('topVideos')
display(topVideos)
print('topVideosMetrics')
display(topVideosMetrics)
print('mostSubscribed')
display(mostSubscribed)
print('mostSubscribedMetrics')
display(mostSubscribedMetrics)
print('mostPopularChannels')
display(mostPopularChannels)
print('mostPopularChannelsMetrics')
display(mostPopularChannelsMetrics)
print('mostPopularVideos')
display(mostPopularVideos)
print('mostPopularVideosMetrics')
display(mostPopularVideosMetrics)

## Top canais que também estão na lista de mais inscritos

In [0]:
display(topChannels.join(mostSubscribed.select('youtuber_id'), topChannels.youtuber_id == mostSubscribed.youtuber_id, 'inner').orderBy('rank'))

## Top canais que não estão na lista de mais inscritos

In [0]:
display(topChannels.join(mostSubscribed.select('youtuber_id'), topChannels.youtuber_id == mostSubscribed.youtuber_id, 'left_anti').orderBy('rank'))

## Canais mais incritos que não são Top canais

In [0]:
display(mostSubscribed.join(topChannels.select('youtuber_id'), topChannels.youtuber_id == mostSubscribed.youtuber_id, 'left_anti').orderBy('rank'))

## Top channels com quantidade de subscribers maior ou igual a média de subscribers da tabela

In [0]:
mean_subscribers = topChannels.select(F.mean(F.col('subscribers'))).collect()[0][0]
display(topChannels.filter(topChannels.subscribers >= mean_subscribers))

## Média de views por vídeo do canal

In [0]:
display(topChannels.withColumn('views/count', F.round(F.col('video_views') / F.col('video_count'))))

## Nome do canal e categoria dos youtubers mais antigos

In [0]:
oldestesChannels = topChannels.orderBy(F.col('started')).limit(10)
display(oldestesChannels.join(youtuberName, oldestesChannels.youtuber_id == youtuberName.youtuber_id, 'inner').select('started', 'youtuber', 'category'))

## Nome do canal e categoria dos youtubers mais recentes

In [0]:
newchannels = topChannels.orderBy(F.col('started').desc()).limit(10)
display(newchannels.join(youtuberName, newchannels.youtuber_id == youtuberName.youtuber_id, 'inner').select('started', 'youtuber', 'category'))